# Preparing Training/Validation Datasets
Adjust as needed for project versions and testing

## Imports

In [ ]:
import os
import pickle

import numpy as np
import librosa as lb

## Parameters

In [ ]:
sampleRate = 44100
channels = 1
repalce = False

gain = ["Low", "Mid", "High"]
context = [44, 66, 88]

effect = "SD-1"
projVer = "DS340"

## Functions

In [ ]:
def normalizeAudio(data):
    maxAll = np.max(np.abs(data))
    return data / maxAll

#use gap, and gapSize experiments later
def splitChunks(dry, wet, context):
    x = []
    y = []
    for i in range(0, len(dry) - context, context):
        seq = dry[i:i+context]
        out = wet[i+context-1]
        x.append(seq)
        y.append(out)
    return x, y

#you don't actually really need this
#might rework this one
def shrinkData(x, y, trainSize, replace = False):
    indexes = np.random.choice(x.shape[0], trainSize, replace = replace)
    return x[indexes], y[indexes]


def loadData(context, gain):
    drys = []
    wets = []
    for _, _, files in os.walk(f"../Data/{effect}/{projVer}/Train/{gain}/Dry"):
        for file in files:
            fileName = file[2:]
            dry, _ = lb.load(f"../Data/{effect}/{projVer}/Train/{gain}/Dry/d_" + fileName, sr=sampleRate, mono=True)
            wet, _ = lb.load(f"../Data/{effect}/{projVer}/Train/{gain}/Wet/w_" + fileName, sr=sampleRate, mono=True)
            x, y = splitChunks(dry, wet, context)
            drys += x
            wets += y
    drys = np.array(drys)
    wets = np.array(wets)

    directory = f"../TrainValPickles/{effect}/{projVer}"
    
    os.makedirs(directory, exist_ok=True)

    with open(f"{directory}/{gain}_{context}_x.pkl", "wb") as file1:
            pickle.dump(drys, file1)

    with open(f"{directory}/{gain}_{context}_y.pkl", "wb") as file2:
            pickle.dump(wets, file2)

    return drys, wets


## Execute

In [ ]:
for lvl in gain:
    for size in context:
        x, y = loadData(size, lvl)
        